In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import leaguestandingsv3
from datetime import datetime, timezone, timedelta
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import boxscoresummaryv3
import sys
from pathlib import Path
import pandas as pd


# Get the parent directory (Datos folder)
current_dir = Path().resolve()  # libretas folder
datos_dir = current_dir.parent  # Datos folder

# Add Datos to path so we can import from Functions
sys.path.insert(0, str(datos_dir))



pd.set_option("display.max_columns", None)


In [2]:

from helpfuncs import get_current_season
import numpy as np
from typing import List, Dict, Tuple, Union, Optional

In [34]:
class Games():
    def __init__(self):
        self.current_season = get_current_season()

    def check_valid_season(self, season: str) -> str:
        """Check if the provided season is valid and return formatted season."""
        if not season:
            season = self.current_season
        elif not season.isdigit():
            raise ValueError(f"Invalid season format: {season}. Expected a year in 'YYYY' format.")
        elif int(season) < 1980 or int(season) > datetime.now().year:
            raise ValueError(f"Invalid season year: {season}. Year must be between 1980 and {datetime.now().year}.")
        else:
            season = f"{season}-{str(int(season)+1)[-2:]}"
        
        return season

    def get_team_game_log(self, team_id: int, season: str) -> pd.DataFrame:
        game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
        game_log = game_log.get_data_frames()[0]
        return game_log

    def get_todays_games(self) -> None:
        f = "{gameId}: {awayTeam} @ {homeTeam} : {gameTimeLTZ}" 
        board = scoreboard.ScoreBoard()
        print("ScoreBoardDate: " + board.score_board_date)
        games = board.games.get_dict()
        for game in games:
            gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
            print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))
        
        return games
    
    def get_game_by_id(self, game_id: str) -> Dict:
        """Get game details by game ID"""
        try:
            
            game_summary = boxscoresummaryv3.BoxScoreSummaryV3(game_id=game_id)
            
            # Get all dataframes
            game_summary_data = game_summary.get_data_frames()[7]
            line_score = game_summary.get_data_frames()[1]
            
            return game_summary_data
            
        except Exception as e:
            print(f"Error retrieving game {game_id}: {e}")
            raise e

    def get_current_standings(self, season: str = None, conference:str = 'Overall') -> pd.DataFrame:
        """Get current NBA standings for all teams"""
        conference_types = ['Overall','West','East']
        try:
            if conference in conference_types:
                season = self.check_valid_season(season)
                
                standings = leaguestandingsv3.LeagueStandingsV3(
                    league_id='00',  # NBA
                    season=season,
                    season_type='Regular Season'
                )
                
                standings_df = standings.get_data_frames()[0]
                
                # Select relevant columns
                columns_to_keep = [
                    'TeamID', 'TeamCity', 'TeamName', 'Conference','WINS', 'LOSSES', 'PlayoffRank', 
                    'Record', "HOME","ROAD", "L10"
                ]
                
                standings_df = standings_df[columns_to_keep]

                if conference != 'Overall':
                    return standings_df[standings_df['Conference'] == conference].reset_index(drop=True)
                else:
                    return standings_df
            else:
                raise ValueError(f"Invalid conference type: {conference}. Expected one of {conference_types}.")    
        except Exception as e:
            print(f"Error retrieving standings for season {season}: {e}")
            raise e
        
    def get_last_week_games(self, team_id: int, season: str = '2025') -> pd.DataFrame:
        # Get the last 7 days date range
        today = datetime.now()
        seven_days_ago = today - timedelta(days=8)


        # Get team game log
        weekly_games = Games().get_team_game_log(team_id=team_id, season=season)

        # Convert GAME_DATE to datetime for comparison
        weekly_games['GAME_DATE'] = pd.to_datetime(weekly_games['GAME_DATE'])

        # Filter for games in the last 7 days
        recent_games = weekly_games[
            (weekly_games['GAME_DATE'] >= seven_days_ago) & 
            (weekly_games['GAME_DATE'] <= today)
        ]

        return recent_games


In [35]:
Games().get_last_week_games(team_id=1610612743)

/var/folders/bt/nvzckywj42s3jj55fjzzjdm80000gn/T/ipykernel_51041/2405768117.py:93: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weekly_games['GAME_DATE'] = pd.to_datetime(weekly_games['GAME_DATE'])


,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612743,0022501225,2025-12-15,DEN vs. HOU,W,19,6,0.76,265,43,85,0.506,15,34,0.441,27,33,0.818,7,31,38,24,10,4,10,28,128
1,1610612743,0022501208,2025-12-11,DEN @ SAC,W,18,6,0.75,240,52,88,0.591,12,29,0.414,20,29,0.690,10,36,46,35,9,0,12,18,136


In [ ]:
#Games().get_game_by_id(game_id='0022500278')
#Games().get_current_standings(season='2025', conference='Overall')
#games = Games().get_todays_games()
#Games().get_game_by_id(game_id='0022501225')

,gameId,teamId,teamCity,teamName,teamTricode,points,reboundsTotal,assists,steals,blocks,turnovers,fieldGoalsPercentage,threePointersPercentage,freeThrowsPercentage,pointsInThePaint,pointsSecondChance,pointsFastBreak,biggestLead,leadChanges,timesTied,biggestScoringRun,turnoversTeam,turnoversTotal,reboundsTeam,pointsFromTurnovers,benchPoints
0,0022501225,1610612743,Denver,Nuggets,DEN,128.0,38.0,24.0,10.0,4.0,10.0,0.506,0.441,0.818,42.0,12.0,15.0,9.0,17.0,16.0,10.0,1.0,11.0,8.0,17.0,25.0
1,0022501225,1610612745,Houston,Rockets,HOU,125.0,46.0,29.0,7.0,8.0,16.0,0.505,0.480,0.731,56.0,19.0,10.0,9.0,17.0,16.0,8.0,0.0,16.0,10.0,12.0,21.0
